<a href="https://colab.research.google.com/github/jordanmsouza/Ac4_container_docker/blob/main/TechChallenge_Fase3_Grupo4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Montando o Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Instalação de dependências:

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install triton
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-uujlg8q7/unsloth_96b5ffd749e3409a9f56bef730b3181e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-uujlg8q7/unsloth_96b5ffd749e3409a9f56bef730b3181e
  Resolved https://github.com/unslothai/unsloth.git to commit 0fbbdfc091fc1a3b1c09b752794963681d10fad2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached xformers-0.0.28.post1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
  Using cached peft-0.12.0-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.43.3-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
Using cached xformers-0.0.28.post1-cp310-cp310-manylinux_2_28_x86_64.whl (16.7 MB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
Using cached peft-0.12.0-py3-none-any.whl (29

### Importando Bibliotecas

In [2]:
import json
import pandas as pd
from datasets import load_dataset
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import pipeline

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### Configurações do modelo para fine-tuning

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

### Modelos Compativeis

In [ ]:
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

### Formatando o Dataset de produtos

In [ ]:
def format_dataset_into_model_input(data):
    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Alterando a formatação para título e descrição
    for example in data:
        title = example['title']
        content = example['content']
        instruction = "Descreva o produto de forma estilizada."

        instructions.append(instruction)
        inputs.append(title)
        outputs.append(content)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON (caso necessário)
    formatted_json_path = "/content/drive/MyDrive/trn.json/formatted_amazon_product_data.json"
    with open(formatted_json_path, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset formatado salvo em: {formatted_json_path}")

    return formatted_json_path

### Declarando Constantes e Formatando o Dataset da Amazon

In [ ]:
# Caminho do arquivo JSON original
DATA_PATH = '/content/drive/MyDrive/trn.json/trn.json'

# Caminho do arquivo CSV que será gerado
OUTPUT_PATH_DATASET = format_dataset_into_model_input(dataset['train'] if 'train' in dataset else dataset)


Dataset formatado salvo em: /content/drive/MyDrive/trn.json/formatted_amazon_product_data.json


### Carregando o modelo pré treinado

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Ajustes finos do LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

### Prompt para a descrição dos produtos

In [ ]:
alpaca_prompt = """Abaixo está uma instrução que descreve a tarefa, juntamente com um título que fornece mais contexto. Escreva uma resposta que conclua a descrição de forma apropriada.

### Instrução:
{}

### Título:
{}

### Descrição:
{}"""

EOS_TOKEN = tokenizer.eos_token

### Função para formatar o Prompt

In [ ]:
def formatting_prompts_func(data):
    instructions = data["instruction"]
    inputs = dataset["input"]
    outputs = data["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

### Carregar o Dataset

In [ ]:
dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET)
formatted_dataset = dataset['train'].map(formatting_prompts_func, batched=True)

In [ ]:
formatted_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 294805
})

In [ ]:
dataset['train'][0]

{'instruction': 'Descreva o produto de forma estilizada.',
 'input': 'Methodical Bible study: A new approach to hermeneutics',
 'output': 'Inductive study compares related Bible texts in order to let the Bible interpret itself, rather than approaching Scripture with predetermined notions of what it will say.  Dr. Trainas Methodical Bible Study was not intended to be the last word in inductive Bible study; but since its first publication in 1952, it has become a foundational text in this field.  Christian colleges and seminaries have made it required reading for beginning Bible students, while many churches have used it for their lay Bible study groups.  Dr. Traina summarizes its success in this comment:  "If the truths of the Bible already resided in man, there would be no need for the Bible and this manual would be superfluous.  But the fact is the Bible is an objective body of literature which exists because man needs to know certain truths which he himself cannot know.  There are tw

### Configurações do Trainer

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,  # Usando o dataset formatado e o split 'train'
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/294805 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


### Treinando o Modelo

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 294,805 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.140400
2,2.324500
3,2.148700
4,1.823100
5,1.593600
6,1.583500
7,1.609100
8,1.462100
9,1.414200
10,1.421800


### Salvando o Modelo

In [ ]:
# Salvar o modelo ajustado
model.save_pretrained("/content/drive/MyDrive/trn.json/lora_model")
tokenizer.save_pretrained("/content/drive/MyDrive/trn.json/lora_model")

('/content/drive/MyDrive/trn.json/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/trn.json/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/trn.json/lora_model/tokenizer.model',
 '/content/drive/MyDrive/trn.json/lora_model/added_tokens.json',
 '/content/drive/MyDrive/trn.json/lora_model/tokenizer.json')

### Reutilizando o modelo já treinado

In [6]:
# Caminho para o modelo salvo
model_path = "/content/drive/MyDrive/trn.json/lora_model"

# Carregar o modelo e o tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(model_path)

# Preparar o modelo para inferência
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.9: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load /content/drive/MyDrive/trn.json/lora_model as a legacy tokenizer.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(

### Função para testar o modelo

In [7]:
def generate_description(title):
    input_text = f"Descreva o produto de forma estilizada.\nTítulo: {title}\nDescrição:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

In [11]:
# Testar com um exemplo
title = "Harry Potter"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

Título: Harry Potter
Descrição: Descreva o produto de forma estilizada.
Título: Harry Potter
Descrição:
Harry Potter and the Sorcerer's Stone is the first book in the Harry Potter series by J. K. Rowling. The book follows Harry Potter, a young wizard, in his first year at Hogwarts School of Witchcraft and Wizardry. Harry discovers that he is a wizard after the Dursleys, his Muggle (non-magical) guardians, neglect to prevent Harry from receiving a letter from Hogwarts. Harry is rescued from the Dursleys by his friend Ron Weasley and taken to Hogw


### Instalando dependências para criação da interface grafica

In [12]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [13]:
import streamlit as st

### Função para carregar o modelo treinado

In [15]:
def load_model():
    model_path = "/content/drive/MyDrive/trn.json/lora_model"
    model, tokenizer = FastLanguageModel.from_pretrained(model_path)
    FastLanguageModel.for_inference(model)
    return model, tokenizer

### Função para gerar a descrição do modelo

In [16]:
def generate_description(model, tokenizer, title):
    input_text = f"Descreva o produto de forma estilizada.\nTítulo: {title}\nDescrição:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

### Criando a interface do Streamlit

In [17]:
st.title("Gerador de Descrições de Produtos")
st.write("Digite o título do produto para gerar uma descrição estilizada:")

2024-09-21 02:15:47.125 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:15:47.310 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-09-21 02:15:47.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:15:47.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:15:47.334 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:15:47.337 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:15:47.339 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


### Entrada do usuário

In [18]:
title_input = st.text_input("Título do Produto")

2024-09-21 02:16:29.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:16:29.545 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:16:29.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:16:29.551 Session state does not function when running a script without `streamlit run`
2024-09-21 02:16:29.554 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:16:29.560 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


### Carregando Modelo

In [19]:
model, tokenizer = load_model()

==((====))==  Unsloth 2024.9: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load /content/drive/MyDrive/trn.json/lora_model as a legacy tokenizer.


### Gerando descrição  ao clicar no botão

In [20]:
if st.button("Gerar Descrição"):
    if title_input:
        description = generate_description(model, tokenizer, title_input)
        st.write("### Descrição Gerada:")
        st.write(description)
    else:
        st.write("Por favor, insira um título de produto.")

2024-09-21 02:18:31.447 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:18:31.454 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:18:31.458 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 02:18:31.460 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


### Executando Interface

In [22]:
%%writefile app.py

Writing app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.147.104.77:8501

  Stopping...
  Stopping...


### Criando tunel com o ngrok

In [28]:
!pip install pyngrok

In [31]:
from pyngrok import ngrok
ngrok.set_auth_token("2mMUyErqoNgY1rw1vPJrs5o9GgJ_5aJGqfWAibnh158WCoNR5")

public_url = ngrok.connect(port=8501, proto="http")
print(f"Streamlit app rodando em: {public_url}")

!streamlit run app.py

PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"}}
